In [1]:
import pandas as pd
import re

In [2]:

data = pd.read_csv('./GSAF5.csv', encoding = "ISO-8859-1")
data.head(30)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [3]:
name_columns=list(data.columns.values)
print(name_columns)

['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']


In [4]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

In [5]:
null_cols = data.isnull().sum()
print(null_cols)
print("the number of registers is " + str(data.shape[0]))

#miro en los que hay nulls para ver con qué trabajar.
#miro tb el número de casos registrados
#La de Age tiene demasiados nulls, asi que decido no trabajar con ella
#Visto que la edad no me funciona me planteo si estudiar el componente de la "persona atacada" tiene sentido, 
#puesto que solo tengo el dato "sexo". Voy a ver si ambos sexos están igualmente representados. 

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64
the number of registers is 5992


In [6]:
a="Values are null in more than 40% registers in"
print(a + str(list(null_cols[null_cols > data.shape[0]/2.5].index)))

Values are null in more than 40% registers in['Age', 'Time', 'Species ', 'Unnamed: 22', 'Unnamed: 23']


In [7]:
#Me doy cuenta de que cuando quiero trabajar con la columna Sex no me permite, el nombre debe llevar espacios..
#Si inspecciono los nombres de las cols, veo que alguna otra tiene espacios, voy a limpiar de espacios
name_columns_corrected=[]
for i in name_columns:
    name_columns_corrected.append(i.strip())
print (name_columns_corrected)

['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']


In [8]:
#sustituir los nombres de las columnas por los corregidos
data.columns =name_columns_corrected
print(data.columns.values)

['Case Number' 'Date' 'Year' 'Type' 'Country' 'Area' 'Location' 'Activity'
 'Name' 'Sex' 'Age' 'Injury' 'Fatal (Y/N)' 'Time' 'Species'
 'Investigator or Source' 'pdf' 'href formula' 'href' 'Case Number.1'
 'Case Number.2' 'original order' 'Unnamed: 22' 'Unnamed: 23']


In [9]:
pd.crosstab(index=data["Sex"],columns="count") 
#Observo que de female solo hay 500 registros, no hay suficiente paridad, descarto este dato para tenerlo en cuenta

col_0,count
Sex,
.,1
F,585
M,4835
M,2
N,1
lli,1


In [10]:
#Mis columnas para hipótesis son: ['Date','Type','Country','Area','Investigator or Source','Activity', 'Fatal (Y/N)']
#Sin embargo, quiero ver si hay datos o no en todas las columnas relevantes
#Debo mirar qué columnas pueden ser relevantes en cuanto a tener datos. 
#'Age', 'Time', 'Species ', 'Unnamed: 22', 'Unnamed: 23' tienen más del 40% nulls.los unnameds probablemente son erróneas, puesto que casi no tienen nada
#los case numbers no son relevantes en este caso.Name no es relevante. Injury no es relevante.
#sex ya he visto que no me sirve
#los que no me sirvan para mi hipótesis y no tengan ningún null tampoco me sirven.
#primero tengo que ver si alguna tiene muchos ceros
#Para ver si puedo quitar year voy a ver si en date siempre está bien el year.
pd.crosstab(index=data["Date"],columns="dates") 

col_0,dates
Date,
10-Jan-2009,1
15-Jun-1937,1
16-Jan-1970,1
22-Jul-2013,1
21-Sep-1908,1
02-Jun-1899,1
03-Feb-1914,1
05-Oct-1985,1
10-Jan-1903,1


In [11]:
#I see that dates are from long time ago, so my hipothesis will be centered on years more than the exact date
#Anyway, the exact date is something to take into account in term of whether it is registered. 
#For example, maybe during world war II there are few registers. Something to look is whether in world war II the registers are fewer
#world war II lo voy a codificar como un año intermedio pero me lo apunto (igual con summer, winter..)
#tengo que tener en cuenta aquellos que no tienen fecha exacta (poner una columna"fecha exacta(Y/N))
#date lo voy a utilizar para cuán registrado está y year para hipótesis. 
#pdf y direcciones web (href) no me sirven puesto que solo sirven para la actualidad, en años previos no hay registro porque no había internet.
#original order no parece relevante. 

In [12]:
'''
Estas son con las que me voy a quedar:
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Fatal (Y/N)                 19
Investigator or Source      15
'''

'\nEstas son con las que me voy a quedar:\nDate                         0\nYear                         0\nType                         0\nCountry                     43\nArea                       402\nLocation                   496\nActivity                   527\nFatal (Y/N)                 19\nInvestigator or Source      15\n'

In [13]:
working_data=data[['Date','Year','Type','Country','Area','Location','Activity','Fatal (Y/N)','Investigator or Source' ]]
#Estas son las columnas con las que quiero trabajar, el siguiente paso es limpiarlas
working_data.head()
#con esta working data empezaré a limpiar mis columnas. 
#de aquellas con las que voy a trbajar, hacer una tabla de frecuencias con cada una para ver que necesitan limpiar.

,Date,Year,Type,Country,Area,Location,Activity,Fatal (Y/N),Investigator or Source
0,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,N,"Orlando Sentinel, 9/19/2016"
1,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,N,"Orlando Sentinel, 9/19/2016"
2,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,N,"Orlando Sentinel, 9/19/2016"
3,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,N,"The Age, 9/18/2016"
4,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,N,"The Age, 9/16/2016"


In [14]:
#empezamos por date. ya hicimos la tabla de frecuencias cuando queríamos saber si incluirla. 
#quitar la palabra reported y mirar si alguna más. 
for i in working_data["Date"][0:10]:
    print(type(i))
#date está en formato string. 
#podemos hacer una columna de cuánto de completa es:
#si está en formato num-str-num es completa
#si está en formato str-num o str num es semi completa
#si está en formato str o pone "Some time between" es incompleta 
#si está en otros poner "otro" o mirar y ver dónde meter

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [27]:
pattern1="^\s*\t*[0-9]+\-[A-Z]?[a-z]+\-[0-9]+" # 00-mm-00
pattern2="^\s*\t*[A-Z]?[a-z]+\-[0-9]+" # mm-00
#pattern3=
#pattern4=


In [28]:
for i in working_data["Date"][0:1000]:
    if not re.findall(pattern1,i) and not re.findall(pattern2,i):
        print(i)

2014
16-Aug--2011
11-Aug--2011
 12-Jan 2011
190Feb-2010


In [18]:
working_data['Date']=working_data['Date'].str.replace('Reported', '')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
p1="^\s*\t*[0-9]+\-[A-Z]?[a-z]+\-[0-9]+" #00-mm-00
p2="^\s*\t*[A-Z]?[a-z]+\-[0-9]+" # mm-00
p3="^\s*\t*[0-9]+" # 0000  only year
#pattern4=

for i in working_data["Date"][0:5000]:
    if not re.findall(p1,i) and not re.findall(p2,i) and not re.findall(p3,i):
        print(i)
#repito el ejercicio para ilustrar mi manera de ver patrones. A partir de aquí lo hago en la sig todo

Late Jul-2008
Fall 2008
Early Aug-2006
Late Jul-2003
Early Sep-2000
Early Jun-2000
Early Jul-1995
Last incident of 1994 in Hong Kong
Late May 1993
Fall 1993
Between May & Nov-1993
Mid Jul-1985 or mid Jul-1986
Ca. 1983
Late Aug-1982
Summer of 1981
Late Jul-1980
Early Jul-1980
Summer 1980
Early Feb-1974
Summer 1974
Late Apr-1971
Late 1970s
Ca. 1970
Winter 1969
Early Nov-1966
Mid Aug-1966
Summer of 1996
May-Jun-1965
Summer 1965
Early 1965
May-Jun-1965
May-Jun-1965
Ca. 1965
Early 1963
Late Aug-1962
Jan-Jun-1962
Ca. 1962
Early summer 1960
Late 1960s
Ca. 1960
Ca. 1960
Between 10 and 12-Sep-1959
Late Jul-1959
Summer of 1959
Jul- to Sep-1959
Circa 1958
Ca. 1955 
Between 01-Aug-1951 & 08-Aug-1951
Summer 1950
Summer 1950
Ca. 1950
Ca. 1950
Summer 1948
Between 18 & 22-Dec 1944
Some time between Apr & Nov-1944
Summer 1943
Fall 1943
Fall 1943
Winter 1942
Summer 1942
Ca. 1940
Woirld War II
Ca. 1939
Ca. 1929
Late Jul-1928
Some time between 08-Jan-1928 & 21-Jan-1928
Summer of 1926


In [ ]:
pattern1="^\s*\t*[0-9]+\-[A-Z]?[a-z]+\-[0-9]+" #00-mm-00
pattern2="^\s*\t*[A-Z]?[a-z]+\-[0-9]+" # mm-00
pattern3="^\s*\t*[0-9]+" # 0000  only year
#pattern4=

for i in working_data["Date"][0:5000]:
    if not re.findall(pattern1,i) and not re.findall(pattern2,i) and not re.findall(pattern3,i):
        print(i)

In [53]:
working_data['Date']=working_data['Date'].str.replace('Ca.', '')
working_data['Date']=working_data['Date'].str.replace('of', '')
working_data['Date']=working_data['Date'].str.replace('Early', '')
working_data['Date']=working_data['Date'].str.replace('Late', '')
working_data['Date']=working_data['Date'].str.replace('Some time between ', '')
working_data['Date']=working_data['Date'].str.replace('Between ', '')
working_data['Date']=working_data['Date'].str.replace('Circa ', '')
#Ignoro los early y los late porque me parece muy subjetivos

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [59]:
pattern1="^\s*\t*[0-9]+\-[A-Z]?[a-z]+\-[0-9]+" #00-mm-00
pattern2="^\s*\t*[A-Z]?[a-z]+\-[0-9]+" # mm-00
pattern3="^\s*\t*[0-9]+" # 0000  only year
pattern41="^Summer\s*[0-9]+"
pattern42="^Winter\s*[0-9]+"
pattern43="^Fall\s*[0-9]+"
pattern44="^Autumn\s*[0-9]+"
pattern5="Before[a-z]?\s*[0-9]+"


for i in working_data["Date"]:
    if not re.findall(pattern1,i) and not re.findall(pattern2,i) and not re.findall(pattern3,i) and not re.findall(pattern41,i) and not re.findall(pattern42,i) and not re.findall(pattern43,i) and not re.findall(pattern44,i) and not re.findall(pattern5,i):
        print(i)
        

Last incident  1994 in Hong Kong
 May 1993
 May & Nov-1993
Mid Jul-1985 or mid Jul-1986
Mid Aug-1966
May-Jun-1965
May-Jun-1965
May-Jun-1965
Jan-Jun-1962
 summer 1960
Jul- to Sep-1959
 Apr & Nov-1944
Woirld War II
 to have taken place in 1919
. 22-Feb-1902
Mid Oct-1901
Reportd 15-Jul-1894
October 1887
Mid-Aug-1886
Mid-Aug-1886
Nov- or Dec-1873
Sep or Oct-1853
"Anniversary Day" 22-Jan-1850 or 1852
 in 1847
. 1825
May 1812
June 1721
Letter dated 10-Jan-1580
Before Oct-2009
No date, late 1960s
No date
No date
Before Feb-1998
No date, Before May-1996
No date, Before Mar-1995
No date, Before Aug-1989
No date, Before Aug-1987
No date, After August 1926 and before 1936
Before Mar-1956
"During the war" 1943-1945
"Before the war"
Said to be 1941-1945, more likely 1945
World War II
World War II
A few years before 1938
No date
No date
No date
No date
No date (3 days after preceding incident) & prior to 19-Jul-1913
